In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras import backend as K

training_data_path = '/Users/luna/Documents/Hand-car-and-face-classifier-with-OpenCV-and-TensorFlow/src/notebooks/hands/photos/train'
validation_data_path = '/Users/luna/Documents/Hand-car-and-face-classifier-with-OpenCV-and-TensorFlow/src/notebooks/hands/photos/test'

K.clear_session()  # Clear everything

# Parameters
iterations = 20  # Number of iterations to adjust our model
height, width = 200, 200  # Size of the training images
batch_size = 1  # Number of images that we will send sequentially
steps = 299 / 1  # Number of times information will be processed in each iteration
validation_steps = 299 / 1  # After each iteration, we validate the previous one
conv1_filters = 32
conv2_filters = 64  # Number of filters that we will apply in each convolution
filter_size1 = (3, 3)
filter_size2 = (2, 2)  # Sizes of filters 1 and 2
pool_size = (2, 2)  # Size of the filter in max pooling
classes = 2  # Open and closed hand (5 fingers and 0 fingers)
lr = 0.0005  # Adjustments of the neural network to approach an optimal solution

# Pre-Processing of the images
training_preprocessing = ImageDataGenerator(
    rescale=1. / 255,  # Scale pixels from 0 to 255 | 0 to 1
    shear_range=0.3,  # Generate our images tilted for better training
    zoom_range=0.3,  # Generates images with zoom for better training
    horizontal_flip=True  # Flip the images for better training
)

validation_preprocessing = ImageDataGenerator(
    rescale=1. / 255
)

training_image = training_preprocessing.flow_from_directory(
    training_data_path,  # It will take the photos that we have already stored
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification = by classes
)

validation_image = validation_preprocessing.flow_from_directory(
    validation_data_path,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create the Convolutional Neural Network (CNN)
cnn = Sequential()  # Sequential neural network
# We add filters in order to make our image very deep but small
cnn.add(Conv2D(conv1_filters, filter_size1, padding='same', input_shape=(height, width, 3), activation='relu'))  # Add the first layer
cnn.add(MaxPooling2D(pool_size=pool_size))  # After the first layer, we will have a max pooling layer and assign the size

cnn.add(Conv2D(conv2_filters, filter_size2, padding='same', activation='relu'))  # Add new layer

cnn.add(MaxPooling2D(pool_size=pool_size))

# Now we will convert that deep image into a flat one, to have 1 dimension with all the info
cnn.add(Flatten())  # Flatten the image
cnn.add(Dense(256, activation='relu'))  # Assign 256 neurons
cnn.add(Dropout(0.5))  # Turn off 50% of the neurons in the previous function to avoid overfitting
cnn.add(Dense(classes, activation='softmax'))  # It's our last layer, it tells us the probability of being one of the classes

# Add parameters to optimize the model
# During training have a self-evaluation, optimize with Adam, and the metric will be accuracy
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
cnn.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# We will train our network
cnn.fit(training_image, steps_per_epoch=steps, epochs=iterations, validation_data=validation_image, validation_steps=validation_steps)

# Save the model
cnn.save('Model.h5')
cnn.save_weights('weights.h5')


2024-02-19 16:53:33.557646: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 135 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Epoch 1/20
299/299 [==============================] - 67s 218ms/step - loss: 0.0050 - accuracy: 1.0000


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
